# Prepare features

Prepare the necessary features and transformations

# Set up

In [1]:
import os
import sys

import datasets
from datasets import load_dataset
from dotenv import load_dotenv
from pydantic import BaseModel
from sqlalchemy import create_engine
import pandas as pd

sys.path.insert(0, "..")

from src.utils.data_prep import handle_dtypes, parse_dt

load_dotenv("../.env")
datasets.logging.set_verbosity_error()

# Controller

In [2]:
class Args(BaseModel):
    run_name: str = "000-prep-data"
    testing: bool = True
    notebook_persist_dp: str = None
    random_seed: int = 41

    train_fp: str = "../../data/train.parquet"
    val_fp: str = "../../data/val.parquet"

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    # Output PostgreSQL table
    table_name: str = "amz_review_rating_raw"

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        if not self.testing:
            os.makedirs(self.notebook_persist_dp, exist_ok=True)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

{
  "run_name": "000-prep-data",
  "testing": true,
  "notebook_persist_dp": "/Users/dvq/frostmourne/recsys-mvp/feature_pipeline/notebooks/data/000-prep-data",
  "random_seed": 41,
  "train_fp": "../../data/train.parquet",
  "val_fp": "../../data/val.parquet",
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "table_name": "amz_review_rating_raw"
}


# Load data

In [3]:
metadata_raw = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Video_Games", trust_remote_code=True
)
metadata_raw_df = metadata_raw["full"].to_pandas()
metadata_raw_df

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Video Games,Dash 8-300 Professional Add-On,5.0,1,[Features Dash 8-300 and 8-Q300 ('Q' rollout l...,[The Dash 8-300 Professional Add-On lets you p...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Aerosoft,"[Video Games, PC, Games]","{""Pricing"": ""The strikethrough price is the Li...",B000FH0MHO,None,None,None
1,Video Games,Phantasmagoria: A Puzzle of Flesh,4.1,18,[Windows 95],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Sierra,"[Video Games, PC, Games]","{""Best Sellers Rank"": {""Video Games"": 137612, ...",B00069EVOG,None,None,None
2,Video Games,NBA 2K17 - Early Tip Off Edition - PlayStation 4,4.3,223,[The #1 rated NBA video game simulation series...,[Following the record-breaking launch of NBA 2...,58.0,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['NBA 2K17 - Kobe: Haters vs Players...,2K,"[Video Games, PlayStation 4, Games]","{""Release date"": ""September 16, 2016"", ""Best S...",B00Z9TLVK0,None,None,None
3,Video Games,Nintendo Selects: The Legend of Zelda Ocarina ...,4.9,22,[Authentic Nintendo Selects: The Legend of Zel...,[],37.42,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Amazon Renewed,"[Video Games, Legacy Systems, Nintendo Systems...","{""Best Sellers Rank"": {""Video Games"": 51019, ""...",B07SZJZV88,None,None,None
4,Video Games,Thrustmaster Elite Fitness Pack for Nintendo Wii,3.0,3,"[Includes (9) Total Accessories, Pedometer, Wi...",[The Thrustmaster Motion Plus Elite Fitness Pa...,None,"{'hi_res': [None, None, None, None, None, None...","{'title': [], 'url': [], 'user_id': []}",THRUSTMASTER,"[Video Games, Legacy Systems, Nintendo Systems...","{""Release date"": ""November 1, 2009"", ""Pricing""...",B002WH4ZJG,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137264,None,Story of Seasons: Pioneers Of Olive Town (Nint...,4.5,397,[A wild world of discovery - tame the wilderne...,"[Product Description, Inspired by Tales of you...",31.04,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Marvelous Europe,"[Video Games, Nintendo Switch, Games]","{""Release date"": ""March 26, 2021"", ""Best Selle...",B09XQJS4CZ,None,None,None
137265,Video Games,MotoGP 18 (PC DVD) UK IMPORT REGION FREE,4.0,1,[Brand new game engine - MotoGP18 has been reb...,[Become the champion of the 2018 MotoGP Season...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Milestone,"[Video Games, Game Genre of the Month]","{""Pricing"": ""The strikethrough price is the Li...",B07DGPTGNV,None,None,None
137266,Cell Phones & Accessories,Century Accessory Soft Silicone Protective Ski...,2.9,19,"[Easy access to all buttons, controls and port...",[This soft case cover will add a splash of col...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Century Accessory,"[Video Games, Legacy Systems, Xbox Systems, Xb...","{""Package Dimensions"": ""2.76 x 2.76 x 0.2 inch...",B00HUWCQBW,None,None,None
137267,None,Hasbro Interactive Mr. Potato Head Activity Pa...,3.9,5,[],"[Amazon.com, Everyone's favorite master-of-dis...",None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Hasbro,"[Video Games, PC, Games]","{""Release date"": ""July 24, 1999"", ""Best Seller...",B00002S9MH,None,None,None


In [4]:
if not os.path.exists(args.train_fp):
    raise Exception(
        f"{args.train_fp} does not exist, you need to run the notebook 000-prep-data in the parent recsys-mvp folder first"
    )

train_df = pd.read_parquet(args.train_fp)
val_df = pd.read_parquet(args.val_fp)

In [5]:
train_df

,user_id,parent_asin,rating,timestamp
54,AHATA6X6MYTC3VNBFJ3WIYVK257A,B0050SVNZ8,4.0,1321885664000
55,AHATA6X6MYTC3VNBFJ3WIYVK257A,B00LZVNWIA,4.0,1408233606000
61,AHATA6X6MYTC3VNBFJ3WIYVK257A,B0BH98D8GL,5.0,1511708554100
62,AHATA6X6MYTC3VNBFJ3WIYVK257A,B074RNL1RX,5.0,1511753174174
63,AHATA6X6MYTC3VNBFJ3WIYVK257A,B089QYP649,5.0,1531092820696
...,...,...,...,...
736763,AE3P3SRQSH7R4R7RR2KMUEWLEXPQ,B004IWRNTC,5.0,1394472136000
736764,AE3P3SRQSH7R4R7RR2KMUEWLEXPQ,B01FSKACPY,5.0,1394472165000
736765,AE3P3SRQSH7R4R7RR2KMUEWLEXPQ,B002JTX9WQ,5.0,1394472180000
736767,AE3P3SRQSH7R4R7RR2KMUEWLEXPQ,B0017QFMJU,5.0,1394472206000


# Merge metadata

In [6]:
cols = ["main_category", "title", "description", "categories", "price"]

# Merge the item features into the interaction data
train_features_df = pd.merge(
    train_df, metadata_raw_df[[args.item_col] + cols], how="left", on=args.item_col
)
val_features_df = pd.merge(
    val_df, metadata_raw_df[[args.item_col] + cols], how="left", on=args.item_col
)
train_features_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price
0,AHATA6X6MYTC3VNBFJ3WIYVK257A,B0050SVNZ8,4.0,1321885664000,Video Games,Amazon Basics Carrying Case for Nintendo - New...,[],"[Video Games, Legacy Systems, Nintendo Systems...",None
1,AHATA6X6MYTC3VNBFJ3WIYVK257A,B00LZVNWIA,4.0,1408233606000,Computers,Logitech G402 Hyperion Fury FPS Gaming Mouse,[Logitech G402 Hyperion Fury FPS Gaming Mouse],"[Video Games, PC, Accessories, Gaming Mice]",None
2,AHATA6X6MYTC3VNBFJ3WIYVK257A,B0BH98D8GL,5.0,1511708554100,Computers,Logitech G433 7.1 Wired Gaming Headset with DT...,[Logitech G433 gaming headset is the premium a...,"[Video Games, Xbox One, Accessories, Headsets]",44.99
3,AHATA6X6MYTC3VNBFJ3WIYVK257A,B074RNL1RX,5.0,1511753174174,Video Games,Razer Wolverine Ultimate Officially Licensed X...,[Play anywhere with the Razer Wolverine Ultima...,"[Video Games, PC, Accessories, Controllers]",64.98
4,AHATA6X6MYTC3VNBFJ3WIYVK257A,B089QYP649,5.0,1531092820696,Video Games,Turtle Beach Stealth 600 Wireless Surround Sou...,[The Turtle Beach Stealth 600 is the latest wi...,"[Video Games, PlayStation 4, Accessories, Head...",168.75
...,...,...,...,...,...,...,...,...,...
164293,AE3P3SRQSH7R4R7RR2KMUEWLEXPQ,B004IWRNTC,5.0,1394472136000,Video Games,You Don't Know Jack - Xbox 360,"[Product Description, The award-winning You Do...","[Video Games, Legacy Systems, Xbox Systems, Xb...",20.55
164294,AE3P3SRQSH7R4R7RR2KMUEWLEXPQ,B01FSKACPY,5.0,1394472165000,Video Games,Wheel of Fortune - Xbox 360,[Spin the wheel along with Pat Sajak and Vanna...,"[Video Games, Legacy Systems, Xbox Systems, Xb...",24.99
164295,AE3P3SRQSH7R4R7RR2KMUEWLEXPQ,B002JTX9WQ,5.0,1394472180000,Video Games,Press Your Luck 2010 Edition - PC,"[Collect ""spins by answering trivia questions,...","[Video Games, PC, Games]",135.0
164296,AE3P3SRQSH7R4R7RR2KMUEWLEXPQ,B0017QFMJU,5.0,1394472206000,Video Games,Tzou AC Power Adapter for Nintendo Wii Console,[Did you misplace or destroy the AC power adap...,"[Video Games, Legacy Systems, Nintendo Systems...",None


In [7]:
full_df = (
    pd.concat(
        [train_features_df, val_features_df],
        axis=0,
    )
    .assign(
        description=lambda df: df["description"].apply(list),
        categories=lambda df: df["categories"].apply(list),
    )
    .pipe(parse_dt)
    .pipe(handle_dtypes)
)
full_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price
0,AHATA6X6MYTC3VNBFJ3WIYVK257A,B0050SVNZ8,4.0,2011-11-21 14:27:44.000,Video Games,Amazon Basics Carrying Case for Nintendo - New...,[],"[Video Games, Legacy Systems, Nintendo Systems...",None
1,AHATA6X6MYTC3VNBFJ3WIYVK257A,B00LZVNWIA,4.0,2014-08-17 00:00:06.000,Computers,Logitech G402 Hyperion Fury FPS Gaming Mouse,[Logitech G402 Hyperion Fury FPS Gaming Mouse],"[Video Games, PC, Accessories, Gaming Mice]",None
2,AHATA6X6MYTC3VNBFJ3WIYVK257A,B0BH98D8GL,5.0,2017-11-26 15:02:34.100,Computers,Logitech G433 7.1 Wired Gaming Headset with DT...,[Logitech G433 gaming headset is the premium a...,"[Video Games, Xbox One, Accessories, Headsets]",44.99
3,AHATA6X6MYTC3VNBFJ3WIYVK257A,B074RNL1RX,5.0,2017-11-27 03:26:14.174,Video Games,Razer Wolverine Ultimate Officially Licensed X...,[Play anywhere with the Razer Wolverine Ultima...,"[Video Games, PC, Accessories, Controllers]",64.98
4,AHATA6X6MYTC3VNBFJ3WIYVK257A,B089QYP649,5.0,2018-07-08 23:33:40.696,Video Games,Turtle Beach Stealth 600 Wireless Surround Sou...,[The Turtle Beach Stealth 600 is the latest wi...,"[Video Games, PlayStation 4, Accessories, Head...",168.75
...,...,...,...,...,...,...,...,...,...
957,AE3NRCMFIBBA2XVODR47YYNLKRDA,B001EYUQC8,5.0,2021-11-13 09:59:46.634,Video Games,007 Quantum Of Solace - Playstation 3,[James Bond is back to settle the score in Qua...,"[Video Games, Legacy Systems, PlayStation Syst...",44.49
958,AEV5TZDZQEP24PM3SZ7SNV4TR26Q,B01N3ASPNV,5.0,2022-06-17 07:42:54.083,All Electronics,amFilm Tempered Glass Screen Protector for Nin...,[],"[Video Games, Nintendo Switch, Accessories, Fa...",8.91
959,AELRDP5MCGSCANM6GWUXAMBN75LQ,B009AGXH64,5.0,2022-06-03 18:23:36.536,Video Games,Nintendo Wii U Console - 32GB Black Deluxe Set,[Wii U is the next great gaming console from N...,"[Video Games, Legacy Systems, Nintendo Systems...",199.99
960,AHERXKLMQLGPQLW4ZLKD4IRLMZAA,B07M6RVMPJ,5.0,2021-11-27 00:36:11.015,Video Games,Mario Party: The Top 100 - Nintendo 3DS,[Ever partied with Mario? stuffed mouthfuls of...,"[Video Games, Legacy Systems, Nintendo Systems...",48.99


# Insert data into PostgreSQL

In [8]:
# PostgreSQL connection details
username = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")
port = os.getenv("POSTGRES_PORT")
database = os.getenv("POSTGRES_DB")
schema = os.getenv("POSTGRES_OLTP_SCHEMA")

# Create a connection string and engine outside the function
connection_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

In [9]:
to_insert_df = full_df.copy()

In [10]:
to_insert_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price
0,AHATA6X6MYTC3VNBFJ3WIYVK257A,B0050SVNZ8,4.0,2011-11-21 14:27:44.000,Video Games,Amazon Basics Carrying Case for Nintendo - New...,[],"[Video Games, Legacy Systems, Nintendo Systems...",None
1,AHATA6X6MYTC3VNBFJ3WIYVK257A,B00LZVNWIA,4.0,2014-08-17 00:00:06.000,Computers,Logitech G402 Hyperion Fury FPS Gaming Mouse,[Logitech G402 Hyperion Fury FPS Gaming Mouse],"[Video Games, PC, Accessories, Gaming Mice]",None
2,AHATA6X6MYTC3VNBFJ3WIYVK257A,B0BH98D8GL,5.0,2017-11-26 15:02:34.100,Computers,Logitech G433 7.1 Wired Gaming Headset with DT...,[Logitech G433 gaming headset is the premium a...,"[Video Games, Xbox One, Accessories, Headsets]",44.99
3,AHATA6X6MYTC3VNBFJ3WIYVK257A,B074RNL1RX,5.0,2017-11-27 03:26:14.174,Video Games,Razer Wolverine Ultimate Officially Licensed X...,[Play anywhere with the Razer Wolverine Ultima...,"[Video Games, PC, Accessories, Controllers]",64.98
4,AHATA6X6MYTC3VNBFJ3WIYVK257A,B089QYP649,5.0,2018-07-08 23:33:40.696,Video Games,Turtle Beach Stealth 600 Wireless Surround Sou...,[The Turtle Beach Stealth 600 is the latest wi...,"[Video Games, PlayStation 4, Accessories, Head...",168.75
...,...,...,...,...,...,...,...,...,...
957,AE3NRCMFIBBA2XVODR47YYNLKRDA,B001EYUQC8,5.0,2021-11-13 09:59:46.634,Video Games,007 Quantum Of Solace - Playstation 3,[James Bond is back to settle the score in Qua...,"[Video Games, Legacy Systems, PlayStation Syst...",44.49
958,AEV5TZDZQEP24PM3SZ7SNV4TR26Q,B01N3ASPNV,5.0,2022-06-17 07:42:54.083,All Electronics,amFilm Tempered Glass Screen Protector for Nin...,[],"[Video Games, Nintendo Switch, Accessories, Fa...",8.91
959,AELRDP5MCGSCANM6GWUXAMBN75LQ,B009AGXH64,5.0,2022-06-03 18:23:36.536,Video Games,Nintendo Wii U Console - 32GB Black Deluxe Set,[Wii U is the next great gaming console from N...,"[Video Games, Legacy Systems, Nintendo Systems...",199.99
960,AHERXKLMQLGPQLW4ZLKD4IRLMZAA,B07M6RVMPJ,5.0,2021-11-27 00:36:11.015,Video Games,Mario Party: The Top 100 - Nintendo 3DS,[Ever partied with Mario? stuffed mouthfuls of...,"[Video Games, Legacy Systems, Nintendo Systems...",48.99


In [11]:
to_insert_df.to_sql(
    args.table_name, engine, if_exists="append", index=False, schema=schema
)

260